Google Colab Setup

In [1]:
!git clone https://github.com/jan1na/Bleeding-Detecting-on-Capsule-Endoscopy.git

%cd Bleeding-Detecting-on-Capsule-Endoscopy
!git checkout "janina"

import sys

sys.path.append('/content/Bleeding-Detecting-on-Capsule-Endoscopy/scripts')

Cloning into 'Bleeding-Detecting-on-Capsule-Endoscopy'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 227 (delta 106), reused 144 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (227/227), 12.98 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/Bleeding-Detecting-on-Capsule-Endoscopy
Branch 'janina' set up to track remote branch 'janina' from 'origin'.
Switched to a new branch 'janina'


Drive Setup

In [2]:
from google.colab import drive

# Mount Google Drive (for data storage)
drive.mount('/content/drive')

Mounted at /content/drive


# Deep Learning Part

## TODO
* losses should be weighted for different classes, there are 8 to 9 times of healthy images than there is for bleeding, so model will be inclined to predict healthy
* hyperparameters for deep learning runs should be saved

## Imports

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.optim import lr_scheduler
from torch.utils.data.sampler import WeightedRandomSampler

from models import MobileNetV2
from bleeding_dataset import BleedDataset

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Training Space

### Hyperparameters

In [4]:
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/runs/"

TRAIN_TEST_SPLIT = (0.8, 0.1)  # remaining parts will be test
DIRECTORY_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/project_capsule_dataset"
BATCH_SIZE = 16
LR = 0.001  # learning rate

NUM_OF_EPOCHS = 20
EARLY_STOP_LIMIT = 3

THRESHOLD = 0.5  # predictions bigger than threshold will be counted as bleeding prediction, and lower ones will be healthy prediction
MODEL = MobileNetV2  # ResNet, AlexNet, VGG19
APPLY_AUGMENTATION = False
USE_COSINE_ANNEALING_LR = True
AUGMENT_TIMES = 8

### Dataset, Model etc. Inıtıalization

In [5]:
### ---|---|---|---|---|---|---|---|---|---|--- MODEL & DATASET ---|---|---|---|---|---|---|---|---|---|--- ###
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()


# Model Initialization
def initialize_model(model_class, save_path):
    model = model_class().to(device)
    model_serial_number = f"training_with_{model.__class__.__name__}_{datetime.now().strftime('on_%m.%d._at_%H:%M:%S')}"
    model_serial_path = os.path.join(save_path, model_serial_number)
    os.makedirs(model_serial_path, exist_ok=True)
    return model, model_serial_path


model, model_serial_path = initialize_model(MODEL, SAVE_PATH)
full_dataset = None


# Dataset Preparation
def prepare_datasets(dataset_class, directory, split_ratios, batch_size, image_mode="RGB", seed=0):
    global full_dataset
    full_dataset = dataset_class(directory, mode=image_mode, apply_augmentation=APPLY_AUGMENTATION,
                                 augment_times=AUGMENT_TIMES)
    total_size = len(full_dataset)
    train_size = int(split_ratios[0] * total_size)
    test_size = int(split_ratios[1] * total_size)
    validation_size = total_size - train_size - test_size

    torch.manual_seed(seed)
    train_dataset, validation_dataset, test_dataset = random_split(full_dataset,
                                                                   [train_size, validation_size, test_size])

    # Oversampling for bleeding images
    train_labels = np.array(full_dataset.get_labels())[train_dataset.indices]
    class_counts = np.bincount(train_labels)  # Count occurrences per class
    class_weights = 1.0 / class_counts  # Inverse class frequency

    weights = [class_weights[label] for label in train_labels]
    sampler = WeightedRandomSampler(weights, len(weights))

    return {
        "train": DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, sampler=sampler),
        "validation": DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4,
                                 pin_memory=True),
        "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    }


data_loaders = prepare_datasets(BleedDataset, DIRECTORY_PATH, TRAIN_TEST_SPLIT, BATCH_SIZE, image_mode="RGB", seed=0)

# Penalty for imbalanced dataset
bleeding_weight = 6161 / (713 * AUGMENT_TIMES) if APPLY_AUGMENTATION else 6161 / 713
weights = torch.tensor([1.0, bleeding_weight]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=weights[1])

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,
                                           T_max=NUM_OF_EPOCHS) if USE_COSINE_ANNEALING_LR else lr_scheduler.StepLR(
    optimizer, step_size=10, gamma=0.1)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 214MB/s]
/content/Bleeding-Detecting-on-Capsule-Endoscopy/scripts/bleeding_dataset.py:35: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  A.ElasticTransform(alp

### Training Loop

In [ ]:
### ---|---|---|---|---|---|---|---|---|---|--- TRAINING ---|---|---|---|---|---|---|---|---|---|--- ###
train_losses, validation_losses = [], []
min_validation_loss = None  # minimum achieved loss on validation dataset, used for early stopping
min_validation_path = None  # path to model checkpoint file
early_stop_step = 0

for epoch in range(NUM_OF_EPOCHS):
    if APPLY_AUGMENTATION:
        full_dataset.enable_augmentation()
    averaged_training_loss = 0

    for batch_idx, (images, labels) in tqdm(enumerate(data_loaders['train']), leave=False):
        images, labels = images.to(device), labels.to(device)

        model = model.train()
        outputs = model(images.type(torch.float))

        float_outputs = outputs[:, 0].type(torch.float)
        float_labels = labels.type(torch.float)
        train_loss = criterion(float_outputs, float_labels)
        averaged_training_loss = averaged_training_loss + train_loss

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # calculating the average loss in this epoch's training loop
    averaged_training_loss = averaged_training_loss / len(data_loaders['train'])

    with torch.no_grad():
        if APPLY_AUGMENTATION:
            full_dataset.disable_augmentation()
        model = model.eval()

        # calculate validation loss
        validation_loss = 0.0
        for validation_images, validation_labels in data_loaders['validation']:
            validation_images, validation_labels = validation_images.to(device), validation_labels.to(device)

            validation_outputs = model(validation_images.type(torch.float))

            float_validation_outputs = validation_outputs[:, 0].type(torch.float)
            float_validation_labels = validation_labels.type(torch.float)
            validation_loss += criterion(float_validation_outputs, float_validation_labels).item()

        # and average the loss over dataset length
        validation_loss /= len(data_loaders['validation'])

    # if this is first validation or a new minimum is achieved
    if min_validation_loss is None or validation_loss < min_validation_loss:
        early_stop_step = 0
        min_validation_loss = validation_loss

        # if there is a checkpoint file, remove it
        if min_validation_path is not None:
            os.remove(min_validation_path)

        # save the new checkpoint file
        min_validation_path = os.path.join(model_serial_path,
                                           "min_validation_loss:" + str(min_validation_loss) + "_epoch:" + str(
                                               epoch) + ".pth")
        torch.save(model, min_validation_path)
    else:
        early_stop_step += 1

    # log the losses, and append to the lists
    print(
        f"Epoch: {epoch + 1} | training loss: {averaged_training_loss.item()} | min validation loss: {min_validation_loss}",
        flush=True)
    train_losses.append(averaged_training_loss.item())
    validation_losses.append(validation_loss)
    scheduler.step()

    # check for early stopping
    if early_stop_step >= EARLY_STOP_LIMIT:
        print("early stopping...")
        break

4it [00:14,  2.83s/it]

In [ ]:
plt.plot(train_losses, color='blue', label='Train Loss')
plt.plot(validation_losses, color='orange', label='Validation Loss')
plt.legend()
plt.savefig(os.path.join(model_serial_path, "losses.png"))

## Testing Space

In [ ]:
### ---|---|---|---|---|---|---|---|---|---|--- TESTING ---|---|---|---|---|---|---|---|---|---|--- ###
loaded_model = torch.load(min_validation_path)
loaded_model = loaded_model.eval()

# class_correct counts how many correct predictions for that label [corrects_for_label_0, corrects_for_label_1]
# class_total counts how many predictions are there for that label [predictions_for_label_0, predictions_for_label_1]
class_correct, class_total = [0, 0], [0, 0]
with torch.no_grad():
    if APPLY_AUGMENTATION:
        full_dataset.disable_augmentation()
    for test_images, test_labels in tqdm(data_loaders['test']):
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        test_outputs = loaded_model(test_images.type(torch.float))

        test_outputs = test_outputs.squeeze().type(torch.float)
        test_outputs[test_outputs >= THRESHOLD] = 1
        test_outputs[test_outputs < THRESHOLD] = 0

        # calculate indices for correct predictions
        correct = (test_outputs == test_labels).squeeze()
        for e, label in enumerate(test_labels):
            # increase the correct prediction count for that label
            class_correct[label] += correct[e].item()
            class_total[label] += 1

In [ ]:
# Total: accuracy for whole dataset
print(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}")
print(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}")
print(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}")

with open(os.path.join(model_serial_path, "accuracy.txt"), 'w') as txt:
    txt.write(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}\n")
    txt.write(
        f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}\n")
    txt.write(
        f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}\n")

torch.cuda.empty_cache()